# Imports

In [ ]:
import import_ipynb

import sys
import time

import matplotlib.pyplot as plt
import mpld3
import math
import numpy as np
import pydot
from IPython.display import HTML, SVG, clear_output, display
from pydrake.all import (Box, DiagramBuilder, DirectCollocation,
                         FiniteHorizonLinearQuadraticRegulatorOptions,
                         HPolyhedron, LinearSystem,
                         LogVectorOutput,
                         MakeFiniteHorizonLinearQuadraticRegulator,
                         MultibodyPlant, MultibodyPositionToGeometryPose, 
                         Parser,PiecewisePolynomial, Point, RigidTransform,
                         RotationMatrix, RollPitchYaw, SceneGraph, Simulator, Solve, Sphere,
                         StartMeshcat, TrajectorySource, Variable, eq,
                         AddMultibodyPlantSceneGraph, ConstantVectorSource,
                         ControllabilityMatrix, DiagramBuilder, Linearize,
                         LinearQuadraticRegulator, MatrixGain,
                         MeshcatVisualizerCpp, MultibodyPlant, Parser,
                         Saturation, SceneGraph, Simulator, StartMeshcat,
                         ToLatex, WrapToSystem, LeafSystem, FindResourceOrThrow, AbstractValue,
                         FramePoseVector, namedview, GeometryInstance)
from pydrake.all import (Box, DiagramBuilder,
                         HPolyhedron, LinearSystem,
                         LogVectorOutput,
                         Sphere,
                         StartMeshcat, 
                         MeshcatVisualizerCpp,SceneGraph, Simulator, StartMeshcat,
                         AddMultibodyPlantSceneGraph, Parser, CoulombFriction, Cylinder, RigidTransform,
                         SpatialInertia, Sphere, UnitInertia, MakeRenderEngineVtk, RenderEngineVtkParams,
                         MeshcatVisualizerParams)
                         
import pydrake.symbolic as symbolic

from pydrake import geometry

from pydrake.examples.quadrotor import (QuadrotorGeometry, QuadrotorPlant,
                                        StabilizingLQRController)

from underactuated import FindResource, running_as_notebook, ManipulatorDynamics
from underactuated.jupyter import AdvanceToAndVisualize
from underactuated.meshcat_utils import draw_points, set_planar_viewpoint
from underactuated.quadrotor2d import Quadrotor2D, Quadrotor2DVisualizer

from utils import (AddShape, getMinMaxBox)

if running_as_notebook:
    mpld3.enable_notebook()

importing Jupyter notebook from utils.ipynb


# 2D Direct Collocation

In [ ]:
def quadrotor2D_dircol(start, goal, obstacle_list, breaks, RRT_guess, show=False):
    """
    ADD DOCSTRING TO TALK ABOUT INPUTS AND OUTPUTS AND STRUCTURE
    """
    plant = Quadrotor2D()
    context = plant.CreateDefaultContext()

    N = 61
    max_dt = 0.5
    min_dt = 0.01
    max_tf = N*max_dt
    dircol = DirectCollocation(plant,
                            context,
                            num_time_samples = N,
                            minimum_timestep = min_dt,
                            maximum_timestep = max_dt)
    prog = dircol.prog()
    state = dircol.state()
     
    dircol.prog() 
    dircol.AddEqualTimeIntervalsConstraints()

    #---------------------------------ADD THRUST LIMIT CONSTRAINT
    thrust_limit = 3.0
    u = dircol.input()
    dircol.AddConstraintToAllKnotPoints(-thrust_limit <= u[0])
    dircol.AddConstraintToAllKnotPoints(u[0] <= thrust_limit)
    dircol.AddConstraintToAllKnotPoints(-thrust_limit <= u[1])
    dircol.AddConstraintToAllKnotPoints(u[1] <= thrust_limit)


    #---------------------------------ADD INITIAL STATE
    initial_x = float(start[0])
    initial_y = float(start[1])
    initial_theta = 0.0
    initial_state = (initial_x, initial_y, initial_theta, 0., 0., 0.)
    prog.AddBoundingBoxConstraint(initial_state, initial_state, dircol.initial_state())


    #---------------------------------ADD FINAL STATE
    final_x = float(goal[0])
    final_y = float(goal[1])
    final_theta = 0.0
    final_state = (final_x, final_y, final_theta, 0., 0., 0.)
    prog.AddBoundingBoxConstraint(final_state, final_state, dircol.final_state())

    #----------------------------------CONSTRAIN THETA
    dircol.AddConstraintToAllKnotPoints(state[2]<=90)
    dircol.AddConstraintToAllKnotPoints(state[2]>=-90)

    #---------------------------------ADD COLLISION AVOIDANCE CONSTRAINT
    for ob in obstacles:
        radius = ob[2] + 0.5
        ob_x = ob[0]
        ob_y = ob[1]
        diff_x = state[0]-ob_x
        diff_y = state[1]-ob_y
        dircol.AddConstraintToAllKnotPoints(((diff_x**2) + (diff_y**2))**0.5 >= radius)


    #----------------------------------RUNNING COST
    R = 10  # Cost on input "effort".
    for i in range(len(u)):
        dircol.AddRunningCost(R*(u[i]**2))

    # FINAL COST
    dircol.AddFinalCost(dircol.time())

    #---------------------------------INITIAL GUESS
    initial_x_trajectory = PiecewisePolynomial.FirstOrderHold(
        list(breaks), RRT_guess.T
    )

    dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_trajectory)
    
    #---------------------------------SOLVE
    result = Solve(prog)
    assert result.is_success()

    #--------------------------------REBUILD TRAJECTORIES
    x_trajectory = dircol.ReconstructStateTrajectory(result)
    time_x = np.linspace(x_trajectory.start_time(), x_trajectory.end_time(), 100)
    x_knots = np.hstack([
        x_trajectory.value(t) for t in np.linspace(x_trajectory.start_time(),
                                                x_trajectory.end_time(), 100)])

    u_trajectory = dircol.ReconstructInputTrajectory(result)
    time_u = np.linspace(u_trajectory.start_time(), u_trajectory.end_time(), 100)
    u_knots = np.hstack([
        u_trajectory.value(t) for t in time_u])
    
    #---------------------------------PLOTTING
    def plot_scene(obstacle_list, start, goal, traj):
        ax = plt.gca()
        for o in obstacle_list:
            circle = plt.Circle((o[0], o[1]), o[2], color='k')
            ax.add_artist(circle)
        plt.axis([bounds[0]-0.5, bounds[1]+0.5, bounds[0]-0.5, bounds[1]+0.5])
        plt.plot(start[0], start[1], "xr", markersize=10)
        plt.plot(goal[0], goal[1], "xb", markersize=10)
        plt.plot(traj[0],traj[1])
        plt.plot()
        plt.legend(('start', 'goal'), loc='upper left')
        plt.gca().set_aspect('equal')
    
    if show:
        plt.figure()
        plot_scene(obstacles, start, goal,[x_knots[0,:], x_knots[1,:]])
        plt.tight_layout()

        plt.figure(2)
        plt.plot(time_u,u_knots[0,:])
        plt.xlabel("time (seconds)")
        plt.ylabel("force1 (Newtons)")
        display(plt.show())

        plt.figure(3)
        plt.plot(time_u,u_knots[1,:])
        plt.xlabel("time (seconds)")
        plt.ylabel("force2 (Newtons)")
        display(plt.show())

    return x_trajectory, u_trajectory

# 2D LQR Stabilization

In [ ]:
def quadrotor2D_lqr(x_trajectory, u_trajectory, obstacle_list, show=False):
    """
    ADD DOCSTRING TO TALK ABOUT INPUTS AND OUTPUTS AND STRUCTURE
    """
    options = FiniteHorizonLinearQuadraticRegulatorOptions()
    options.x0 = x_trajectory
    options.u0 = u_trajectory

    #-------------------------------SETUP
    builder = DiagramBuilder()
    plant = builder.AddSystem(Quadrotor2D())
    context = plant.CreateDefaultContext()
    Q = np.diag([1.,1.,1.,1.,1.,1.])
    options.Qf = Q
    regulator = builder.AddSystem(
        MakeFiniteHorizonLinearQuadraticRegulator(plant,
                                                  context,
                                                  t0=options.u0.start_time(),
                                                  tf=options.u0.end_time(),
                                                  Q=Q,
                                                  R=np.eye(2),
                                                  options=options,))
    
    builder.Connect(regulator.get_output_port(0), plant.get_input_port(0))
    builder.Connect(plant.get_output_port(0), regulator.get_input_port(0))
    input_logger = LogVectorOutput(regulator.get_output_port(0), builder)
    state_logger = LogVectorOutput(plant.get_output_port(0), builder)


    #---------------------------Quadrotor 2D Visualizer (object moves out of window)
    visualizer = builder.AddSystem(
        Quadrotor2DVisualizer(show=False))

    # SET AXES AND PLACE OBSTACLES
    visualizer.ax.set_xlim(-5,15)
    visualizer.ax.set_ylim(-5,15)
    for o in obstacle_list:
        circle = plt.Circle((o[0], o[1]), o[2], color='k')
        visualizer.ax.add_artist(circle)


    builder.Connect(plant.get_output_port(0),
                    visualizer.get_input_port(0))
 
    
    #--------------------------- SIMULATION (a few random states)
    diagram = builder.Build()
    simulator = Simulator(diagram)
    simcontext = simulator.get_mutable_context()

    for i in range(3):
        rand = 0.5*np.random.randn(6,)
        start_state = np.array([start[0],start[1],0.,0.,0.,0.]) + rand
        print(start_state)
        simcontext.SetTime(0.)
        simcontext.SetContinuousState(start_state)
        simulator.Initialize()
        AdvanceToAndVisualize(simulator, visualizer,
                          options.u0.end_time() if running_as_notebook else 0.1)
    
    #----------------------------------PLOTTING    
    if show:
        fig, ax = plt.subplots(4,1)
        ax[0].plot(options.u0.get_segment_times(),
                options.u0.vector_values(options.u0.get_segment_times())[0].T)
        input_log = input_logger.FindLog(simulator.get_context())
        ax[0].plot(input_log.sample_times(), input_log.data()[0].T)
        ax[0].legend(('u1 nominal','u1 actual'))

        ax[1].plot(options.u0.get_segment_times(),
                options.u0.vector_values(options.u0.get_segment_times())[1].T)
        input_log = input_logger.FindLog(simulator.get_context())
        ax[1].plot(input_log.sample_times(), input_log.data()[1].T)
        ax[1].legend(('u1 nominal','u1 actual'))

        ax[2].plot(options.x0.get_segment_times(),
                options.x0.vector_values(options.x0.get_segment_times())[0].T, 'b')
        state_log = state_logger.FindLog(simulator.get_context())
        ax[2].plot(state_log.sample_times(), state_log.data()[0].T, 'g')
        ax[2].legend(('x nominal','x actual'))

        ax[3].plot(options.x0.get_segment_times(),
                options.x0.vector_values(options.x0.get_segment_times())[1].T, 'b')
        state_log = state_logger.FindLog(simulator.get_context())
        ax[3].plot(state_log.sample_times(), state_log.data()[0].T, 'g')
        ax[3].legend(('y nominal','y actual'))

        def plot_scene(figure,obstacle_list, start, goal, traj):
            ax = plt.gca()
            for o in obstacle_list:
                circle = plt.Circle((o[0], o[1]), o[2], color='k')
                ax.add_artist(circle)
            plt.axis([bounds[0]-0.5, bounds[1]+0.5, bounds[0]-0.5, bounds[1]+0.5])
            plt.plot(start[0], start[1], "xr", markersize=10)
            plt.plot(goal[0], goal[1], "xb", markersize=10)
            plt.plot(traj[0],traj[1])
            plt.plot()
            plt.legend(('start', 'goal'), loc='upper left')
            plt.gca().set_aspect('equal')
        
        fig2 = plt.figure(figsize=(5,5))
        plot_scene(fig2,obstacle_list,start,goal, [options.x0.vector_values(options.x0.get_segment_times())[0].T,
                                                options.x0.vector_values(options.x0.get_segment_times())[1].T])
        plt.plot(state_log.data()[0].T,state_log.data()[1].T, 'g-')


        display(plt.show())


# 3D Direct Collocation

In [ ]:
def quadrotor3D_dircol(start, goal, obstacle_list,show=False):
    """
    TODO: Add Plotting at the end to show paths from random perturbations
    TODO: Add constraint for box objects (check len(object) in object list; 4 = sphere, 6 = box)

    Inputs: start = [x, y, z, phi, theta, psi, xdot, ydot, zdot, phidot, thetadot, psidot] INITIAL
            goal = [x, y, z, phi, theta, psi, xdot, ydot, zdot, phidot, thetadot, psidot]  FINAL
            obstacle_list = list of spherical or box obstacles
            show = True/False to show or hide plots
            
    Outputs: x_trajectory = knot points for state-space
             u_trajectory = knot points for input
    """
    plant = QuadrotorPlant()
    context = plant.CreateDefaultContext()
    N = 61
    max_dt = 0.5
    min_dt = 0.01
    max_tf = N*max_dt
    dircol = DirectCollocation(plant,
                            context,
                            num_time_samples = N,
                            minimum_timestep = min_dt,
                            maximum_timestep = max_dt)
    prog = dircol.prog()
    state = dircol.state()


    # STATE  = [x, y, z, roll, pitch, yaw].T
     
    dircol.prog() 
    dircol.AddEqualTimeIntervalsConstraints()

    #---------------------------------ADD THRUST LIMIT CONSTRAINT
    thrust_limit = 10.0
    u = dircol.input()
    dircol.AddConstraintToAllKnotPoints(-thrust_limit <= u[0])
    dircol.AddConstraintToAllKnotPoints(u[0] <= thrust_limit)
    #dircol.AddConstraintToAllKnotPoints(-thrust_limit <= u[1])
    #dircol.AddConstraintToAllKnotPoints(u[1] <= thrust_limit)
    dircol.AddConstraintToAllKnotPoints(-thrust_limit <= u[2])
    dircol.AddConstraintToAllKnotPoints(u[2] <= thrust_limit)
    #dircol.AddConstraintToAllKnotPoints(-thrust_limit <= u[3])
    #dircol.AddConstraintToAllKnotPoints(u[3] <= thrust_limit)

    #----------------------------------CONSTRAIN ROLL, PITCH, YAW
    
    dircol.AddConstraintToAllKnotPoints(state[4]<= (np.pi/2 - 0.1))
    dircol.AddConstraintToAllKnotPoints(state[4]>=-(np.pi/2 - 0.1))

    #---------------------------------ADD Initial States
    initial_x = float(start[0])
    initial_y = float(start[1])
    initial_z = float(start[2])
    initial_state = (initial_x, initial_y, initial_z, 0., 0., 0., 0., 0., 0., 0., 0., 0.)
    prog.AddBoundingBoxConstraint(initial_state, initial_state, dircol.initial_state())


    #---------------------------------ADD Final States
    final_x = float(goal[0])
    final_y = float(goal[1])
    final_z = float(goal[2])
    final_state = (final_x, final_y, final_z, 0., 0., 0., 0., 0., 0., 0., 0., 0.)
    prog.AddBoundingBoxConstraint(final_state, final_state, dircol.final_state())

    #---------------------------------ADD COLLISION AVOIDANCE CONSTRAINT
    for ob in obstacle_list:
        radius = ob[3] + 1 #rework tolerance based on feasibility
        ob_x = ob[0]
        ob_y = ob[1]
        ob_z = ob[2]
        diff_x = state[0]-ob_x
        diff_y = state[1]-ob_y
        diff_z = state[2]-ob_z
        dircol.AddConstraintToAllKnotPoints(((diff_x**2) + (diff_y**2) + (diff_z**2))**0.5 >= radius)

    #----------------------------------Running COST
    R = 10  # Cost on input "effort".
    for i in range(len(u)):
        dircol.AddRunningCost(R*(u[i]**2))

    # FINAL COST
    dircol.AddFinalCost(dircol.time())

    #-------------------------------- INITIAL GUESS
    initial_x_trajectory = PiecewisePolynomial.FirstOrderHold(
        list(breaks), RRT3D_guess.T)

    dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_trajectory)
    
    #---------------------------------SOLVE
    result = Solve(prog)
    assert result.is_success()

    #---------------------------------REBUILD TRAJECTORIES
    x_trajectory = dircol.ReconstructStateTrajectory(result)
    time_x = np.linspace(x_trajectory.start_time(), x_trajectory.end_time(), 100)
    x_knots = np.hstack([
        x_trajectory.value(t) for t in np.linspace(x_trajectory.start_time(),
                                                x_trajectory.end_time(), 100)])

    u_trajectory = dircol.ReconstructInputTrajectory(result)
    time_u = np.linspace(u_trajectory.start_time(), u_trajectory.end_time(), 100)
    u_knots = np.hstack([
        u_trajectory.value(t) for t in time_u])
    
    #--------------------------------- PLOT
    def plot_scene(obstacle_list, start, goal, traj):
        ax = plt.gca()
        for o in obstacle_list:
            circle = plt.Circle((o[0], o[1]), o[2], color='k')
            ax.add_artist(circle)
        plt.axis([bounds[0]-0.5, bounds[1]+0.5, bounds[0]-0.5, bounds[1]+0.5])
        plt.plot(start[0], start[1], "xr", markersize=10)
        plt.plot(goal[0], goal[1], "xb", markersize=10)
        plt.plot(traj[0],traj[1])
        plt.plot()
        plt.legend(('start', 'goal'), loc='upper left')
        plt.gca().set_aspect('equal')
    
    if show:
        plt.figure()
        plot_scene(obstacles, start, goal,[x_knots[0,:], x_knots[2,:]])
        plt.tight_layout()

        plt.figure(2)
        plt.plot(time_u,u_knots[0,:])
        plt.xlabel("time (seconds)")
        plt.ylabel("force1 (Newtons)")
        #display(plt.show())

        plt.figure(3)
        plt.plot(time_u,u_knots[1,:])
        plt.xlabel("time (seconds)")
        plt.ylabel("force2 (Newtons)")
        #display(plt.show())

        plt.figure(4)
        plt.plot(time_u,u_knots[2,:])
        plt.xlabel("time (seconds)")
        plt.ylabel("force3 (Newtons)")
        #display(plt.show())

        plt.figure(5)
        plt.plot(time_u,u_knots[3,:])
        plt.xlabel("time (seconds)")
        plt.ylabel("force4 (Newtons)")
        #display(plt.show())

    return x_trajectory, u_trajectory

# 3D LQR Stabilization

In [ ]:
def quadrotor3D_lqr(start, x_trajectory, u_trajectory, meshcat):
    '''
    Inputs: start = [x, y, z, phi, theta, psi, xdot, ydot, zdot, phidot, thetadot, psidot]
            x_trajectory = array output of trajectory optimization
            u_trajectory = array output of trajectory optimization
            meshcat = simulation link 
            
    Goal:   1. input a trajectory for feedback stabilization with LQR(t) over the trajectory 
            2. simulate in Meshcat
    '''
    options = FiniteHorizonLinearQuadraticRegulatorOptions()
    options.x0 = x_trajectory
    options.u0 = u_trajectory

    builder = DiagramBuilder()
    plant = builder.AddSystem(QuadrotorPlant())
    context = plant.CreateDefaultContext()
    Q = np.diag([10.,10.,10.,10.,10.,10.,1.,1.,1.,1.,1.,1.])
    options.Qf = Q

    #----------------------------MAKE REGULATOR AND CONNECT
    regulator = builder.AddSystem(
        MakeFiniteHorizonLinearQuadraticRegulator(plant,
                                                  context,
                                                  t0=options.u0.start_time(),
                                                  tf=options.u0.end_time(),
                                                  Q=Q,
                                                  R=np.eye(4),
                                                  options=options))
    builder.Connect(regulator.get_output_port(0), plant.get_input_port(0))
    builder.Connect(plant.get_output_port(0), regulator.get_input_port(0))
    input_logger = LogVectorOutput(regulator.get_output_port(0), builder)
    state_logger = LogVectorOutput(plant.get_output_port(0), builder)

    #---------------------------MESHCAT VISUALIZATION
    scene_graph = builder.AddSystem(SceneGraph())
    QuadrotorGeometry.AddToBuilder(builder, plant.get_output_port(0), scene_graph)

    mbp = builder.AddSystem(MultibodyPlant(time_step=0.0))
    mbp.RegisterAsSourceForSceneGraph(scene_graph)

    for i, obs in enumerate(obstacles):
        x,y,z,r = obs
        obstacle = AddShape(mbp, geometry.Sphere(r), f"obstacle_{i}", color = [1,0,0,1])
        mbp.WeldFrames(mbp.world_frame(), mbp.GetFrameByName(f"obstacle_{i}"),
                       RigidTransform(p=[x, y, z]))

    # 
    builder.Connect(
        mbp.get_geometry_poses_output_port(),
        scene_graph.get_source_pose_port(mbp.get_source_id()),
    )
    builder.Connect(
        scene_graph.get_query_output_port(),
        mbp.get_geometry_query_input_port(),
    )
    mbp.Finalize()
        
    meshcat.Delete()
    meshcat.ResetRenderMode()
    meshcat.SetProperty('/Background','visible',False)
    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
    # end setup for visualization

    
   #----------------------------SIMULATOR
    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
    simcontext = simulator.get_mutable_context()
    quad_context = plant.GetMyContextFromRoot(simcontext)

    visualizer.StartRecording(False)
    for i in range(3):
        rand = 0.5*np.random.randn(12,)
        start_state = np.array([start3D[0],start3D[1],start3D[2],0.,0.,0.,0.,0.,0.,0.,0.,0.]) + rand
        simcontext.SetTime(0.)
        quad_context.SetContinuousState(start_state)
        simulator.Initialize()
        simulator.AdvanceTo(x_trajectory.end_time())
    visualizer.PublishRecording()
    


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=931e8205-9a5c-4b8e-a3da-167564211c03' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>